In [4]:
import warnings
warnings.filterwarnings('ignore')
%reload_ext autoreload
%autoreload 2
from vision_ai_api import APIVisionAI
from IPython.display import Markdown
import pandas as pd
import io


## API

In [41]:
import requests
import time
import json


class APIVisionAIForm(APIVisionAI):
    
    def put_camera_object(self, camera_id, object_id, label_explanation, label):
        return self._put(
            path=f"/cameras/{camera_id}/objects/{object_id}?label={label}&label_explanation={label_explanation}",

        )

with open("secrets.json") as f:
    s = json.load(f)


vision_ai_api = APIVisionAIForm(
    username=s.get("username"),
    password=s.get("password"),
)


def get_objects_table_from_sheets(
    url: str = "https://docs.google.com/spreadsheets/d/122uOaPr8YdW5PTzrxSPF-FD0tgco596HqgB7WK7cHFw/edit#gid=1672006844",
):
    request_url = url.replace("edit#gid=", "export?format=csv&gid=")
    response = requests.get(request_url)
    dataframe = pd.read_csv(io.StringIO(response.content.decode("utf-8")), dtype=str)
    dataframe["label"] = dataframe["label"].fillna("null")
    dataframe = dataframe[dataframe['use'] == '1']
    dataframe = dataframe.drop(columns=['use'])
  
    return dataframe

In [55]:
## GET ALL active cameras

cameras_ativas = vision_ai_api._get_all_pages("/agents/89173394-ee85-4613-8d2b-b0f860c26b0f/cameras")
cameras_ativas_ids = [f"/cameras/{d.get('id')}" for d in cameras_ativas]
cameras_agent = vision_ai_api._get_all_pages(cameras_ativas_ids)

Getting all pages for /agents/89173394-ee85-4613-8d2b-b0f860c26b0f/cameras
Page /agents/89173394-ee85-4613-8d2b-b0f860c26b0f/cameras?page=2&size=100 out 2 took 0.08 seconds
Page /agents/89173394-ee85-4613-8d2b-b0f860c26b0f/cameras?page=1&size=100 out 2 took 0.1 seconds
Getting all pages done!!!
Page /cameras/001490 out 96 took 0.24 seconds
Page /cameras/001487 out 96 took 0.24 seconds
Page /cameras/001160 out 96 took 0.28 seconds
Page /cameras/001511 out 96 took 0.29 seconds
Page /cameras/000801 out 96 took 0.23 seconds
Page /cameras/001381 out 96 took 0.27 seconds
Page /cameras/001538 out 96 took 0.24 seconds
Page /cameras/000869 out 96 took 0.26 seconds
Page /cameras/001495 out 96 took 0.31 seconds
Page /cameras/001167 out 96 took 0.28 seconds
Page /cameras/001171 out 96 took 0.38 seconds
Page /cameras/001530 out 96 took 0.44 seconds
Page /cameras/000528 out 96 took 0.41 seconds
Page /cameras/001484 out 96 took 0.45 seconds
Page /cameras/001391 out 96 took 0.35 seconds
Page /cameras/

In [87]:
## GET ALL OBJECTS

objects = vision_ai_api._get_all_pages('/objects')
object_slug = "image_description"
object_parameters = vision_ai_api.get_item_from_slug(
    slug = object_slug,
    data = objects
)

Getting all pages for /objects
Page /objects?page=1&size=100 out 1 took 0.16 seconds
Getting all pages done!!!


In [106]:
# ### PUT CAMERA LABELS

# object_parameters = vision_ai_api.get_item_from_slug(
#     slug = object_slug,
#     data = objects
# )

# camera_id = '000528'
# object_id = "9130fd1e-e3ae-4a7f-aff0-c755f21fadde"
# label = 'null'
# label_explanation = "test"

# r = vision_ai_api._put(
#             f"/cameras/{camera_id}/objects/{object_id}?label={label}&label_explanation={label_explanation}"  
#         )
# r.json()

In [44]:
object_parameters

{'id': '42c86fbc-79c2-4c0e-baec-e04ed0268a5a',
 'name': 'image_condition',
 'slug': 'image_condition',
 'labels': [{'id': 'face66c3-9c13-464c-b2ef-9df54ae5f867',
   'value': 'clean',
   'criteria': 'The image is clear and free of visual interferences.',
   'identification_guide': 'Ensure the image is free from blurring, distortion.'},
  {'id': 'f3f5456a-49a5-4d9b-87cf-d953e86ccb41',
   'value': 'poor',
   'criteria': 'The image has visual interference affecting clarity.',
   'identification_guide': 'Inspect the image for any blurring, overexposure, underexposure, distortion, glitches, or large areas of uniform color.'}]}

In [50]:
# ### PUT OBJECTS LABELS FROM A GOOGLE SHEETS OBJECTS TABLE

# import io
# import pandas as pd



# df = get_objects_table_from_sheets()


# objects = vision_ai_api._get_all_pages('/objects')

# for label in df.to_dict(orient='records'):
#     object_slug = label.get('object')
#     criteria = label.get('criteria')
#     identification_guide = label.get('identification_guide')
#     labels = label.get('label')
#     object_parameters = vision_ai_api.get_item_from_slug(
#         slug = object_slug,
#         data = objects
#     )
#     object_id = object_parameters.get('id')
#     for label_api in object_parameters.get('labels'):
#         if label_api.get("value") == labels:
#             label_id = label_api.get("id")
#             json_data = {
#                   "value": label_api.get("value"),
#                   "criteria": criteria,
#                   "identification_guide": identification_guide
#             }
#             r = vision_ai_api._put(
#                 path=f'/objects/{object_id}/labels/{label_id}', 
#                 json_data=json_data
#             )
            
#             print(json.dumps(r.json(), indent=4))


In [11]:
# # ## PUT LABELS
# object_slug = "road_blockade"
# object_parameters = vision_ai_api.get_item_from_slug(
#     slug = object_slug,
#     data = objects
# )
# object_id = object_parameters.get('id')
# label_id = "e523e091-0a4a-4ede-b5b7-c4ee39003e8e"
# json_data ={
#       "value": "low_indifferent",
#       "criteria": "Estimate the volume of water present on the road, categorizing it as low_indifferent if water level it is less than one-fourth of the wheel of a passenger vehicle.",
#       "identification_guide": "Look for dry, or slightly wet road surfaces and low water level."
#     }

# r = vision_ai_api._put(
#     path=f'/objects/{object_id}/labels/{label_id}', 
#     json_data=json_data
# )
# print(r.json())

In [12]:
# # ## POST LABELS
# object_slug = "water_level"
# object_parameters = vision_ai_api.get_item_from_slug(
#     slug = object_slug,
#     data = objects
# )
# object_id = object_parameters.get('id')
# json_data ={
#       "value": "flodding",
#       "criteria": "Estimate the volume of water present on the road, categorizing it as flooding if the water level is more than half of the wheel of a passenger vehicle.",
#       "identification_guide": "Identify a scenarios where the road is entirely submerged/flooded."
#     }

# r = vision_ai_api._post(
#     path=f'/objects/{object_id}/labels', 
#     json_data=json_data
# )
# print(r)

In [ ]:

# ### CREATE OBJECT


# response = vision_ai_api._post(
#     path = f"/objects/",
#     json_data={
#         "name":"water_level",
#         "slug": "water_level"
#     }
# )
# print(response)

In [10]:

### DELETE OBJECT LABEL

# object_id = "9130fd1e-e3ae-4a7f-aff0-c755f21fadde"
# label_id = "84ea4602-a850-4ed4-839b-0635590a89b4"

# response = vision_ai_api._delete(
#     path = f"/objects/{object_id}/labels/{label_id}"
# )
# print(response.json())

In [9]:
# # ### DELETE PROMPT OBJECTS
# objects = vision_ai_api._get_all_pages('/objects')
# prompts =  vision_ai_api._get_all_pages('/prompts')
# use_objects = ['image_description', 'road_blockade', 'water_in_road', 'image_condition', 'water_level']
# for _object in objects:
#     object_slug = _object.get('slug')
#     object_id = _object.get('id')
#     if object_slug not in use_objects:
#         for prompt in prompts:
#             prompt_id = prompt.get('id')
#             r = vision_ai_api._delete(
#                 path=f'/prompts/{prompt_id}/objects/{object_id}'
#             )
#             print(f"{prompt_id} | {object_slug} | {object_id}")
#             print(r,"\n\n")

Getting all pages for /objects
Page /objects?page=1&size=100 out 1 took 0.21 seconds
Getting all pages done!!!


In [53]:
# objects

In [58]:
# ### POST OBJECTS TO CAMERAS
# objects = vision_ai_api._get_all_pages('/objects')
# use_objects = [ 'traffic_ease_vehicle']
# for _object in objects:
#     object_slug = _object.get('slug')
#     object_id = _object.get('id')
#     if object_slug in use_objects:
#         for camera in cameras_agent:
#             camera_id = camera.get('id')
#             r = vision_ai_api._post(
#                 path=f'/cameras/{camera_id}/objects?object_id={object_id}'
#             )
#             print(f"{camera_id} | {object_slug} | {object_id}")
#             print(r,"\n\n")
            
            
            
            
            

In [59]:
# ## DELETE MULTIPLE OBJECTS FROM CAMERAS

# cameras = vision_ai_api._get_all_pages('/cameras')
# use_objects = [ 'image_condition', 'water_in_road', 'road_blockade', 'image_description']
# for camera in cameras:
#     if camera.get('objects') != []:
#         for object_slug in camera.get('objects'):
#             if object_slug not in use_objects:
#                 object_parameters = vision_ai_api.get_item_from_slug(
#                     slug = object_slug,
#                     data = objects
#                 )
#                 object_id = object_parameters.get('id')
#                 camera_id = camera.get("id")
#                 response = vision_ai_api._delete(
#                     path = f"/cameras/{camera_id}/objects/{object_id}"
                    
#                 )
#                 print(response.json())

In [57]:
# ### POST OBJECTS TO PROMPT

# prompt_parameters = vision_ai_api._get('/prompts').get('items')[0]
# prompt_id = prompt_parameters.get('id')
# objects = vision_ai_api._get_all_pages('/objects')

# use_objects = [ 'traffic_ease_vehicle']
# for _object in objects:
#     object_slug = _object.get('slug')
#     object_id = _object.get('id')
#     if object_slug in use_objects:
#         r = vision_ai_api._post(
#             path=f'/prompts/{prompt_id}/objects?object_id={object_id}'
#         )
#         print(r,"\n\n")

Getting all pages for /objects
Page /objects?page=1&size=100 out 1 took 0.17 seconds
Getting all pages done!!!
{'id': 'ec717601-2bd7-4e9a-ae32-b1076e20a3f4', 'name': 'traffic_ease_vehicle', 'slug': 'traffic_ease_vehicle', 'labels': [{'id': 'aab955ff-207d-4565-a069-d9fdc735aa57', 'value': 'easy', 'criteria': 'Minimal or no water on the road.', 'identification_guide': 'Look for clear, dry, or slightly wet road surfaces and small, manageable puddles that do not interfere with traffic'}, {'id': '500c5931-c2fb-477e-83f2-25fbc69c006f', 'value': 'moderate', 'criteria': 'Presence of significant, larger puddles', 'identification_guide': 'Detect larger puddles that could cause minor traffic disruptions but are still navigable for most vehicles.'}, {'id': 'fff8ea64-a177-4a41-87e2-4cfa809b620d', 'value': 'difficult', 'criteria': 'A partial portion of the road is covered with medium water level', 'identification_guide': 'Identify areas where water coverage is extensive and high, causing notable hin

In [133]:
### PUT PROMPT


prompt_text = """
Role: Urban Road Image Analyst

Objective:

Analyze CCTV images from Rio de Janeiro, focusing solely on road-related elements to assess urban road conditions. Begin with 'image_condition' and follow with 'image_description'.

Constraints:

- Visible Data Only: Limit analysis to what is visibly present on the road. Acknowledge any image quality limitations affecting road analysis.

- Objective Observation: Avoid assumptions and inferences about areas outside the road. Focus exclusively on verifiable road-related details.


Input:

- A CCTV image.


Process:


    1. Image Condition Assessment: Evaluate the image quality, specifically noting issues like blurring, distortion,artifacts, or large areas of uniform color.
    2. Scene Description: Provide an objective description of the road, detailing visible elements related to the road surface and immediate surroundings.
    3. Analysis and Classification:  Use the 'Objects Table' to classify each objects uniquely. Each classification should be supported with clear, observable evidence.
    4. Output: Present findings in a structured JSON format as specified in the output schema. This format is essential for inputting data into an API.

Output:

- Format the output as a JSON instance following the provided schema. 

Output Schema:


```json
{output_schema}
```

Example Output:

- For each object analyzed, your output should resemble the following format:


```json
{output_example}
```

Objects Table:

{objects_table_md}
"""


prompt_parameters = vision_ai_api._get('/prompts').get('items')[0]
prompt_id = prompt_parameters.get('id')
request_body = {
  "name": prompt_parameters.get("name"),
  "model": prompt_parameters.get("model"),
  "prompt_text": prompt_text,
  "max_output_token": prompt_parameters.get("max_output_token"),
  "temperature": prompt_parameters.get("temperature"),
  "top_k": prompt_parameters.get("top_k"),
  "top_p": prompt_parameters.get("top_p")
}


r = vision_ai_api._put(
    path=f'/prompts/{prompt_id}',
    json_data=request_body
)
print(json.dumps(r.json(),  indent=2))

{
  "id": "e25cabfd-6861-47a8-8ff1-3e3efb34ca57",
  "name": "base",
  "model": "gemini-pro-vision",
  "prompt_text": "\nRole: Urban Road Image Analyst\n\nObjective:\n\nAnalyze CCTV images from Rio de Janeiro, focusing solely on road-related elements to assess urban road conditions. Begin with 'image_condition' and follow with 'image_description'.\n\nConstraints:\n\n- Visible Data Only: Limit analysis to what is visibly present on the road. Acknowledge any image quality limitations affecting road analysis.\n\n- Objective Observation: Avoid assumptions and inferences about areas outside the road. Focus exclusively on verifiable road-related details.\n\n\nInput:\n\n- A CCTV image.\n\n\nProcess:\n\n\n    1. Image Condition Assessment: Evaluate the image quality, specifically noting issues like blurring, distortion,artifacts, or large areas of uniform color.\n    2. Scene Description: Provide an objective description of the road, detailing visible elements related to the road surface and im